In [32]:
import tensorflow as tf
from tensorflow import keras
from keras import layers, models
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Activation, Conv2, Dense, Dropout
from tensorflow.keras.optimizers import Adagrad, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import tensorflow_hub as hub

import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy

from PIL import Image


# データを正方形にリサイズするときの辺の長さ
square_length = 1080

# 画像を置いてるパス
image_path = "../image"

# ラベル付け(0春 1夏 2冬)
# TODO: ? フォルダ名に数字入れておくとかのほうがいいかもしれない？
# TODO: 3つ以上に増やす場合は、class_modeやlossをbinaryからcategricalに変える
def get_label(value):
    if value == "Ariel":
        return 0
    elif value == "Other":
        return 1
    
# ラベル更新したら一緒に更新する
def max_label():
    return 2

SyntaxError: invalid syntax (<ipython-input-32-e0a268b7e9b4>, line 62)

In [ ]:
# 顔・瞳検出 https://note.nkmk.me/python-opencv-face-detection-haar-cascade/
face_cascade_path = 'D:/python/OpenCV/data/haarcascades/haarcascade_frontalface_default.xml'
eye_cascade_path = 'D:/python/OpenCV/data/haarcascades/haarcascade_eye.xml'

face_cascade = cv2.CascadeClassifier(face_cascade_path)
eye_cascade = cv2.CascadeClassifier(eye_cascade_path)

left_eye_cascade_path = 'D:/python/OpenCV/data/haarcascades/haarcascade_lefteye_2splits.xml'
right_eye_cascade_path = 'D:/python/OpenCV/data/haarcascades/haarcascade_righteye_2splits.xml'
left_eye_cascade = cv2.CascadeClassifier(left_eye_cascade_path)
right_eye_cascade = cv2.CascadeClassifier(right_eye_cascade_path)


# 瞳だけトリミングした画像を保存する
eye_path = image_path + "/eye"
# if not(os.path.exists(eye_path)):
# もうeyeフォルダあったら処理しない
# なかったらeyeフォルダ作る
# os.mkdir(eye_path)

# 画像全部見る
for file in glob.glob(image_path + "/**/*", recursive=True):
    if os.path.splitext(file)[1] in [".jpg", ".png", ".bmp"]:
        # cv2.imreadが日本語ファイル名に対応していないので、一旦Pillowで読み込んでから変換する
        # https://qiita.com/derodero24/items/f22c22b22451609908ee
        src = np.array(Image.open(file), dtype=np.uint8)
        if src.ndim == 2:  # モノクロ
            pass
        elif src.shape[2] == 3:  # カラー
            src = cv2.cvtColor(src, cv2.COLOR_RGB2BGR)
        elif src.shape[2] == 4:  # 透過
            src = cv2.cvtColor(src, cv2.COLOR_RGBA2BGRA)       
        src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

        faces = face_cascade.detectMultiScale(src_gray)

        # 保存先パス周りの整備(フォルダなかったら作る)
        dst_path = (eye_path + file.replace(image_path,"")).replace(os.sep,"/")
        os.makedirs(os.path.dirname(dst_path), exist_ok=True)
        split_dst_path = os.path.splitext(dst_path)
        
        # 顔→瞳を検出
        for x, y, w, h in faces:
            cv2.rectangle(src, (x, y), (x + w, y + h), (255, 0, 0), 2)
            face = src[y: y + h, x: x + w]
            face_gray = src_gray[y: y + h, x: x + w]
#             eyes = eye_cascade.detectMultiScale(face_gray)
            eye_count = 0
#             for (ex, ey, ew, eh) in eyes:
#                 # 瞳1つずつトリミング
#                 eye = src[y + ey: y + ey + eh, x + ex: x + ex + ew]
                
#                 # 保存
#                 each_dst_path = split_dst_path[0] + "_" + str(eye_count) + split_dst_path[1]
#                 cv2.imwrite(each_dst_path, eye)
                
#                 eye_count += 1
            left_eyes = left_eye_cascade.detectMultiScale(face_gray)
            for (ex, ey, ew, eh) in left_eyes:
                # 瞳1つずつトリミング
                eye = src[y + ey: y + ey + eh, x + ex: x + ex + ew]
                
                # 保存
                each_dst_path = split_dst_path[0] + "_" + str(eye_count) + split_dst_path[1]
                cv2.imwrite(each_dst_path, eye)
                
                eye_count += 1
            right_eyes = right_eye_cascade.detectMultiScale(face_gray)
            for (ex, ey, ew, eh) in right_eyes:
                # 瞳1つずつトリミング
                eye = src[y + ey: y + ey + eh, x + ex: x + ex + ew]
                
                # 保存
                each_dst_path = split_dst_path[0] + "_" + str(eye_count) + split_dst_path[1]
                cv2.imwrite(each_dst_path, eye)
                
                eye_count += 1
#                 cv2.rectangle(face, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)

print("end")

In [ ]:
# 画像の読み込み
datagen = ImageDataGenerator(
    rescale=1./255
#     validation_split=0.1
    )

train_generator = datagen.flow_from_directory(
    "../image/train",
    target_size=(square_length, square_length),
    class_mode='binary',
    batch_size=32,
    subset='training',
)

In [3]:
# モデルの構築
model = keras.Sequential([
#     keras.layers.Conv2D(32, (8,8), activation="relu"),
#     keras.layers.Conv2D(32, (8,8), activation="relu"),
#     keras.layers.MaxPooling2D(pool_size=(4,4)),
#     keras.layers.Flatten(),
#     keras.layers.Dropout(0.2),
#     keras.layers.Dense(200, activation="relu"),
#     keras.layers.Dropout(0.4),
#     keras.layers.Dense(200, activation="relu"),
#     keras.layers.Dropout(0.4),
#     keras.layers.Dense(1),
#     keras.layers.Activation("softmax")
    hub.KerasLayer(
        "https://tfhub.dev/tensorflow/efficientnet/b4/feature-vector/1",
        trainable=False,
    ),
    keras.layers.Dense(1, activation="sigmoid")
])
# memo: shapeで警告出るのは、keras.layers.reshape((...)),imput_shape=(...))を使えば良さそう

# # モデルをコンパイル
model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=0.001), metrics=["accuracy"])
# 学習を実行
model.fit(train_generator, steps_per_epoch=1, epochs=30)
print(model.summary())

Epoch 1/30
1/1 [==============================] - 47s 47s/step - loss: 0.7145 - accuracy: 0.5000
Epoch 2/30
1/1 [==============================] - 36s 36s/step - loss: 0.6928 - accuracy: 0.4375
Epoch 3/30
1/1 [==============================] - 36s 36s/step - loss: 0.6898 - accuracy: 0.5312
Epoch 4/30
1/1 [==============================] - 36s 36s/step - loss: 0.6523 - accuracy: 0.7188
Epoch 5/30
1/1 [==============================] - 37s 37s/step - loss: 0.6314 - accuracy: 0.6562
Epoch 6/30
1/1 [==============================] - 36s 36s/step - loss: 0.6428 - accuracy: 0.5938
Epoch 7/30
1/1 [==============================] - 27s 27s/step - loss: 0.4951 - accuracy: 0.8750
Epoch 8/30
1/1 [==============================] - 36s 36s/step - loss: 0.5558 - accuracy: 0.7188
Epoch 9/30
1/1 [==============================] - 27s 27s/step - loss: 0.6401 - accuracy: 0.5833
Epoch 10/30
1/1 [==============================] - 36s 36s/step - loss: 0.5143 - accuracy: 0.7500
Epoch 11/30
1/1 [============

In [4]:
# テスト用ディレクトリの画像でチェック。正解率を表示する。
total = 0.
ok_count = 0.

test_path = "../image/test"
for dir in os.listdir(test_path):
    if dir == ".DS_Store":
        continue

    dir_path = test_path + "/" + dir
    label = get_label(dir)

    for file in os.listdir(dir_path):
        if file != ".DS_Store":
            file_path = dir_path + "/" + file
            image = np.array(Image.open(file_path).resize(size=(square_length, square_length)))
#             image = image.transpose(2, 0, 1)
#             image = image.reshape(image.shape[0] * image.shape[1] * image.shape[2], 1)
            result = model.predict(np.array([image / 255.]))
            print(result)
#             print("label:", label, "result:", result[0])

#             total += 1.

#             if label == result[0]:
#                 ok_count += 1.

# print("seikai: ", ok_count / total * 100, "%")

[[0.46749255]]
[[0.7382952]]
[[0.43541092]]
[[0.6447168]]
[[0.74514014]]
[[0.7324718]]
[[0.6442177]]
[[0.5168108]]
